In [1]:
import pyspark
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
NESSIE_URI = "http://nessie:19120/api/v1"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"


 
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
        .set('spark.sql.catalog.nessie.s3.endpoint', 'http://minio:9000')
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
  		#MINIO CREDENTIALS
        .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
)

## Start Spark Session     
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

## LOAD A CSV INTO AN SQL VIEW
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
csv_df.createOrReplaceTempView("csv_open_2023")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS nessie.df_open_2023_lesson3 USING iceberg PARTITIONED BY (countryOfOriginCode) AS SELECT * FROM csv_open_2023 ORDER BY countryOfOriginCode;").show()

## ALTER PARTITIONING BASED ON TRUNCATE FIRST LETTER OF firstName
spark.sql("ALTER TABLE nessie.df_open_2023_lesson3 ADD PARTITION FIELD truncate(1, firstName)").show()

## REWRITE ALL THE DATA SO IT ALL USES UPDATED PARTITONING
spark.sql("CALL nessie.system.rewrite_data_files('df_open_2023_lesson3')").show()



:: loading settings :: url = jar:file:/home/docker/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/docker/.ivy2/cache
The jars for the packages stored in: /home/docker/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-35cf405e-8420-4a40-a7c2-f40f9f2fa293;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.3.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12;0.67.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central

25/03/04 11:24:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/04 11:24:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/04 11:24:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/03/04 11:24:52 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
Spark Running


In [2]:

spark.sql("SELECT * FROM nessie.df_open_2023_lesson3").show()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


+------------+-------------------+-------------+------------+------+------+-------------------+-------------------+--------+----------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|     competitorName|    firstName|    lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+-------------------+-------------+------------+------+------+-------------------+-------------------+--------+----------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      455113|      Harmeet Singh|      Harmeet|       Singh|   ACT|     M|                 IN|              India|      28|      Asia|      19739|        CrossFit BFG| 37|180 cm| 78 kg|      16500|       92381|      18|2023|
|     1116202|    Himanshu Mittal|     Himanshu|      Mittal|   ACT|     M|                 IN| 

In [16]:
query = """ CREATE VIEW Persons2023 AS
SELECT competitorId, firstName, lastName, status, gender
FROM nessie.df_open_2023_lesson3
ORDER BY competitorId;
"""

spark.sql(query).show()

++
||
++
++



In [17]:
spark.sql("SELECT * FROM Persons2023").show()

+------------+---------+-----------+------+------+
|competitorId|firstName|   lastName|status|gender|
+------------+---------+-----------+------+------+
|      100002|  Richard|       Neal|   ACT|     M|
|     1000024|    Karen|     Nanney|   ACT|     F|
|      100005|  Matthew|    DiMarco|   ACT|     M|
|     1000057|     Dave|   Drouault|   ACT|     M|
|      100007|   Robson|      Silva|   ACT|     M|
|     1000071|     Olli|Graf-Potter|   ACT|     M|
|     1000090|     Rick|    Ramirez|   ACT|     M|
|     1000116| Jonathan|     Perrin|   ACT|     M|
|     1000121|Alexandra|    Germain|   ACT|     F|
|      100013|     Eric|      Karls|   ACT|     M|
|     1000139|  Mattias|     Thegel|   ACT|     M|
|     1000142|   Bernie|       Kale|   ACT|     M|
|     1000166|    Brian|     Jestel|   ACT|     M|
|     1000168|  Gourdon|    Mathieu|   ACT|     M|
|     1000179|     Will|     Hodder|   ACT|     M|
|     1000187|   Adrian| Van Poppel|   ACT|     M|
|      100019| Giannina|      D

In [13]:
query = """CREATE TABLE IF NOT EXISTS nessie.Males USING iceberg AS
 SELECT *
FROM Persons
WHERE gender='M';
"""

spark.sql(query).show()

++
||
++
++



In [14]:
csv2_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2022.csv")
csv2_df.createOrReplaceTempView("csv_open_2022")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS nessie.df_open_2022_lesson3 USING iceberg PARTITIONED BY (countryOfOriginCode) AS SELECT * FROM csv_open_2022 ORDER BY countryOfOriginCode;").show()


++
||
++
++



In [18]:
query = """ CREATE VIEW Persons2022 AS
SELECT competitorId, firstName, lastName, status, gender
FROM nessie.df_open_2022_lesson3
ORDER BY competitorId;
"""

spark.sql(query).show()

++
||
++
++



In [24]:
query = """CREATE VIEW IF NOT EXISTS both1 AS
SELECT Persons2022.*
FROM Persons2022 Inner JOIN Persons2023
ON Persons2022.lastName = Persons2022.lastName;
"""

spark.sql(query).show()

++
||
++
++



In [25]:
spark.sql("SELECT * FROM both1").show()

+------------+---------+--------+------+------+
|competitorId|firstName|lastName|status|gender|
+------------+---------+--------+------+------+
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|     F|
|     1000007|   Valade|  Sophie|   ACT|

In [28]:
query = """CREATE VIEW IF NOT EXISTS both2 AS
SELECT Persons2022.firstName AS fname, Persons2023.lastName AS lname, Persons2023.* 
FROM Persons2022, Persons2023;
"""

spark.sql(query).show()

++
||
++
++



In [29]:
spark.sql("SELECT * FROM both2").show()

+------+------+------------+---------+-----------+------+------+
| fname| lname|competitorId|firstName|   lastName|status|gender|
+------+------+------------+---------+-----------+------+------+
|Valade|Sophie|      100002|  Richard|       Neal|   ACT|     M|
|Valade|Sophie|     1000024|    Karen|     Nanney|   ACT|     F|
|Valade|Sophie|      100005|  Matthew|    DiMarco|   ACT|     M|
|Valade|Sophie|     1000057|     Dave|   Drouault|   ACT|     M|
|Valade|Sophie|      100007|   Robson|      Silva|   ACT|     M|
|Valade|Sophie|     1000071|     Olli|Graf-Potter|   ACT|     M|
|Valade|Sophie|     1000090|     Rick|    Ramirez|   ACT|     M|
|Valade|Sophie|     1000116| Jonathan|     Perrin|   ACT|     M|
|Valade|Sophie|     1000121|Alexandra|    Germain|   ACT|     F|
|Valade|Sophie|      100013|     Eric|      Karls|   ACT|     M|
|Valade|Sophie|     1000139|  Mattias|     Thegel|   ACT|     M|
|Valade|Sophie|     1000142|   Bernie|       Kale|   ACT|     M|
|Valade|Sophie|     10001

In [30]:
spark.sql("SELECT * FROM Persons2022").show()

+------------+---------+--------+------+------+
|competitorId|firstName|lastName|status|gender|
+------------+---------+--------+------+------+
|     1000007|   Valade|  Sophie|   ACT|     F|
|      100002|  Richard|    Neal|   ACT|     M|
|     1000024|    Karen|  Nanney|   ACT|     F|
|     1000045|   Sinead|Mc Auley|   ACT|     F|
|      100005|  Matthew| DiMarco|   ACT|     M|
|     1000057|     Dave|Drouault|   ACT|     M|
|      100007|   Robson|   Silva|   ACT|     M|
|     1000078|   Rodney|Pedersen|   ACT|     M|
|     1000090|     Rick| Ramirez|   ACT|     M|
|     1000092|     Jorn|Posthuma|   ACT|     M|
|     1000093|  Kristen|   Horan|   ACT|     F|
|     1000116| Jonathan|  Perrin|   ACT|     M|
|     1000119|   Daniel|  Tanner|   ACT|     M|
|     1000120|    Shawn|Soutiere|   ACT|     M|
|      100013|     Eric|   Karls|   ACT|     M|
|     1000139|  Mattias|  Thegel|   ACT|     M|
|     1000160|  Coralie|  Veyret|   ACT|     F|
|     1000166|    Brian|  Jestel|   ACT|

In [31]:
query = """CREATE VIEW IF NOT EXISTS both4 AS
SELECT Persons2022.firstName AS fname, Persons2023.competitorId AS cId, Persons2023.gender AS gen , Persons2023.* 
FROM Persons2022, Persons2023;
"""

spark.sql(query).show()

++
||
++
++



In [32]:
spark.sql("SELECT * FROM both3").show()

+--------+-----+------------+---------+--------+------+------+
|   fname|lname|competitorId|firstName|lastName|status|gender|
+--------+-----+------------+---------+--------+------+------+
|  Valade| Neal|      100002|  Richard|    Neal|   ACT|     M|
| Richard| Neal|      100002|  Richard|    Neal|   ACT|     M|
|   Karen| Neal|      100002|  Richard|    Neal|   ACT|     M|
|  Sinead| Neal|      100002|  Richard|    Neal|   ACT|     M|
| Matthew| Neal|      100002|  Richard|    Neal|   ACT|     M|
|    Dave| Neal|      100002|  Richard|    Neal|   ACT|     M|
|  Robson| Neal|      100002|  Richard|    Neal|   ACT|     M|
|  Rodney| Neal|      100002|  Richard|    Neal|   ACT|     M|
|    Rick| Neal|      100002|  Richard|    Neal|   ACT|     M|
|    Jorn| Neal|      100002|  Richard|    Neal|   ACT|     M|
| Kristen| Neal|      100002|  Richard|    Neal|   ACT|     M|
|Jonathan| Neal|      100002|  Richard|    Neal|   ACT|     M|
|  Daniel| Neal|      100002|  Richard|    Neal|   ACT|

In [ ]:
query = """CREATE VIEW IF NOT EXISTS both4 AS
SELECT Persons2022.firstName AS fname, Persons2022.lastName AS lname, Persons2022. AS lname, Persons2023.* 
FROM Persons2022, Persons2023;
"""

spark.sql(query).show()

In [33]:
query = """CREATE VIEW IF NOT EXISTS both5 AS
SELECT Persons2022.firstName AS fname, Persons2022.lastName AS lname, Persons2022.competitorId AS cId, Persons2022.gender AS gen , Persons2023.* 
FROM Persons2022, Persons2023
WHERE Persons2022.lastName = Persons2023.lastName;
"""

spark.sql(query).show()

++
||
++
++



In [34]:
spark.sql("SELECT * FROM both5").show()

+----------+--------+-------+---+------------+----------+--------+------+------+
|     fname|   lname|    cId|gen|competitorId| firstName|lastName|status|gender|
+----------+--------+-------+---+------------+----------+--------+------+------+
|      John| /Awaseb|2249340|  M|     2249321|Khomtatite| /Awaseb|   ACT|     M|
|Khomtatite| /Awaseb|2249321|  M|     2249321|Khomtatite| /Awaseb|   ACT|     M|
|    Stephy|     A M|1596436|  F|     1596436|    Stephy|     A M|   ACT|     F|
|    Stephy|     A M|1596436|  F|     1056680|    Tobias|     A M|   ACT|     M|
|    Tobias|     A M|1056680|  M|     1596436|    Stephy|     A M|   ACT|     F|
|    Tobias|     A M|1056680|  M|     1056680|    Tobias|     A M|   ACT|     M|
|     Holly|  ACourt|2245127|  F|     2245127|     Holly|  ACourt|   ACT|     F|
|     Claus|  Aagren| 236035|  M|      236035|     Claus|  Aagren|   ACT|     M|
|  Kristine|   Aakre| 935647|  F|     2515505|      Lily|   Aakre|   ACT|     F|
|  Kristine|   Aakre| 935647

In [36]:
 query = """CREATE VIEW IF NOT EXISTS both6 AS
SELECT Persons2022.* AS 2020, Persons2023.* 
FROM Persons2022, Persons2023
WHERE Persons2022.lastName = Persons2023.lastName;
"""

spark.sql(query).show()

ParseException: 
Syntax error at or near '2020'(line 2, pos 24)

== SQL ==
CREATE VIEW IF NOT EXISTS both6 AS
SELECT Persons2022.* AS 2020, Persons2023.* 
------------------------^^^
FROM Persons2022, Persons2023
WHERE Persons2022.lastName = Persons2023.lastName;


In [43]:
query = """CREATE VIEW IF NOT EXISTS both7 AS
SELECT Persons2022.firstName AS fname, Persons2022.lastName AS lname, Persons2022.competitorId AS cId, Persons2022.gender AS gen , Persons2023.*,
CASE WHEN Persons2022.gender = 'M' THEN  'Male' ELSE 'Female' END AS value_desc
FROM Persons2022, Persons2023
WHERE Persons2022.competitorId = Persons2023.competitorId;
"""

spark.sql(query).show()

++
||
++
++



In [44]:
spark.sql("SELECT * FROM both7").show()

+------------+----------+-------+---+------------+---------+----------+------+------+----------+
|       fname|     lname|    cId|gen|competitorId|firstName|  lastName|status|gender|value_desc|
+------------+----------+-------+---+------------+---------+----------+------+------+----------+
|        Sara|  Svensson|1000312|  F|     1000312|     Sara|  Svensson|   ACT|     F|    Female|
|     Natasha|  Hamilton|1000369|  F|     1000369|  Natasha|  Hamilton|   ACT|     F|    Female|
|      Edmond|  Martinez|1000503|  M|     1000503|   Edmond|  Martinez|   ACT|     M|      Male|
|      Marina|Hennebique|1000619|  F|     1000619|   Marina|Hennebique|   ACT|     F|    Female|
|     Melissa|   Dudlets|1000952|  F|     1000952|  Melissa|   Dudlets|   ACT|     F|    Female|
|      Regina|  Petersen|  10010|  F|       10010|   Regina|  Petersen|   ACT|     F|    Female|
|       Aaron|    Toombs|1001065|  M|     1001065|    Aaron|    Toombs|   ACT|     M|      Male|
|       Jason|   Howarth|10011

In [55]:
query = """CREATE OR REPLACE VIEW  both10 AS
SELECT Persons2022.firstName AS fname, Persons2022.lastName AS lname, Persons2022.competitorId AS cId, Persons2022.gender AS gen , Persons2023.*,
CASE WHEN Persons2022.gender = 'M' THEN  'Male' ELSE 'Female' END AS value_desc
FROM Persons2022, Persons2023
WHERE Persons2022.competitorId = Persons2023.competitorId;
"""

spark.sql(query).show()

++
||
++
++



In [58]:
spark.sql("SELECT * FROM both5 limit 5").show()

25/03/04 18:11:53 ERROR BaseReader: Error reading file(s): s3a://warehouse/df_open_2023_lesson3_de53b2d9-7c6f-47ee-8f89-6b8d67984b12/data/countryOfOriginCode=PE/firstName_trunc_1=W/00105-130-fc165b2f-b61f-4ac2-8d97-be5a41885968-00078.parquet
software.amazon.awssdk.core.exception.SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.
	at software.amazon.awssdk.core.exception.SdkClientException$BuilderImpl.build(SdkClientException.java:102)
	at software.amazon.awssdk.awscore.interceptor.HelpfulUnknownHostExceptionInterceptor.modifyException(HelpfulUnknownHostExceptionInterceptor.java:59)
	at software.amazon.awssdk.core.interceptor.ExecutionInterceptorChain.modifyException(ExecutionInterceptorChain.java:199)
	at software.amazon.a

Py4JJavaError: An error occurred while calling o192.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 72.0 failed 1 times, most recent failure: Lost task 9.0 in stage 72.0 (TID 2054) (f0aff0cda6e8 executor driver): software.amazon.awssdk.core.exception.SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.
	at software.amazon.awssdk.core.exception.SdkClientException$BuilderImpl.build(SdkClientException.java:102)
	at software.amazon.awssdk.awscore.interceptor.HelpfulUnknownHostExceptionInterceptor.modifyException(HelpfulUnknownHostExceptionInterceptor.java:59)
	at software.amazon.awssdk.core.interceptor.ExecutionInterceptorChain.modifyException(ExecutionInterceptorChain.java:199)
	

ckObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/0f/temp_shuffle_db0ee5bc-9ff9-4a6e-aaf0-52ad0bf53804
25/03/04 18:12:00 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/38/temp_shuffle_13fa69b1-b851-4083-bca8-3f2fbc6ffd49
25/03/04 18:12:00 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/15/temp_shuffle_236186fd-88b5-4ba6-9047-5d8af99f8e4e
25/03/04 18:12:00 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/16/temp_shuffle_c7d59467-30cb-4111-a61b-dcfba4f5aa88
25/03/04 18:12:00 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/33/temp_shuffle_0c087446-c21a-4921-8683-00a4fdc9b573
25/03/04 18:12:00 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/17/temp_shuffle_3bc2b599-f02c-40e1-9ff7-1beec1aa8cfb
25/03/04 18:12:00 WARN DiskBlockObjectWr

unner.$anonfun$run$3(Executor.scala:548)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1504)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:551)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)
Caused by: software.amazon.awssdk.core.exception.SdkClientException: Unable to execute HTTP request: minio
	at software.amazon.awssdk.core.exception.SdkClientException$BuilderImpl.build(SdkClientException.java:102)
	at software.amazon.awssdk.core.exception.SdkClientException.create(SdkClientException.java:47)
	at software.amazon.awssdk.core.internal.http.pipeline.stages.utils.RetryableStageHelper.setLastException(RetryableStageHelper.java:204)
	at software.amazon.awssdk.core.internal.http.pipeline.stages.RetryableStage.execute(RetryableStage.java:83)
	at software.amazo

sdk.awscore.interceptor.HelpfulUnknownHostExceptionInterceptor.modifyException(HelpfulUnknownHostExceptionInterceptor.java:59)
	at software.amazon.awssdk.core.interceptor.ExecutionInterceptorChain.modifyException(ExecutionInterceptorChain.java:199)
	at software.amazon.awssdk.core.internal.http.pipeline.stages.utils.ExceptionReportingUtils.runModifyException(ExceptionReportingUtils.java:54)
	at software.amazon.awssdk.core.internal.http.pipeline.stages.utils.ExceptionReportingUtils.reportFailureToInterceptors(ExceptionReportingUtils.java:38)
	at software.amazon.awssdk.core.internal.http.pipeline.stages.ExecutionFailureExceptionReportingStage.execute(ExecutionFailureExceptionReportingStage.java:39)
	at software.amazon.awssdk.core.internal.http.pipeline.stages.ExecutionFailureExceptionReportingStage.execute(ExecutionFailureExceptionReportingStage.java:26)
	at software.amazon.awssdk.core.internal.http.AmazonSyncHttpClient$RequestExecutionBuilderImpl.execute(AmazonSyncHttpClient.java:193)
	a

25/03/04 18:12:00 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/24/temp_shuffle_ad9b8803-7585-4155-9e0e-79888b887c9d
25/03/04 18:12:18 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/2f/temp_shuffle_572cbab0-453a-41de-a16d-5d78f22e2002
25/03/04 18:12:18 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/2e/temp_shuffle_9d33f87f-1e42-47f4-b66b-5f4d27067b83
25/03/04 18:12:18 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/1d/temp_shuffle_42f3d9ee-5c29-4de9-ac88-d487f3be1645
25/03/04 18:12:48 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/20/temp_shuffle_37fe18fe-bc90-4eca-819c-7cb960023ddf
25/03/04 18:12:48 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-de310a34-a2e7-484d-b69f-82b0c731d0f1/38/temp_shuffle_6b00de35-8064-4981-b58e-da3d20d4a89f
25/03/04 1

In [ ]:
query = """CREATE TABLE IF NOT EXISTS both10 AS
SELECT Persons2022.firstName AS fname, Persons2022.lastName AS lname, Persons2022.competitorId AS cId, Persons2022.gender AS gen , Persons2023.*,
CASE WHEN Persons2022.gender = 'M' THEN  'Male' ELSE 'Female' END AS value_desc
FROM Persons2022, Persons2023
WHERE Persons2022.competitorId = Persons2023.competitorId;
"""

spark.sql(query).show()